# JUPYTER NOTEBOOK FOR HOMEWORK 2 - question 1
Extracting texture features and matching them for scene classification

In [ ]:
!pip install pandas
!pip install tqdm
!pip install -U scikit-learn
!pip install -U scikit-image

     |████████████████████████████████| 6.8MB 2.7MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1
     |████████████████████████████████| 12.4MB 334kB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: scikit-image 0.16.2
    Uninstalling scikit-image-0.16.2:
      Successfully uninstalled scikit-image-0.16.2


In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks')

Mounted at /content/drive


In [ ]:
import os, glob, time, sys
import pandas as pd
from tqdm import tqdm
import csv
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            
from skimage import io, transform
from skimage.filters.edges import convolve
from skimage.color import rgb2gray

import numpy as np
import joblib
import matplotlib.pyplot as plt

from sklearn.cluster import MiniBatchKMeans, KMeans, AgglomerativeClustering
from sklearn.metrics import classification_report
from sklearn.preprocessing import MinMaxScaler #use for data normalization

from makeLMfilters import makeLMfilters
np.random.seed(42)

## 1. Load the training and testing images from the two .CSV files respectively

In [ ]:
#Load the data along with the class labels
# Note, you will need to use the file folder name to determine the label of an image
# Convert the images to grayscale and resize them to 128 x 128
# (Optional) You might want to save your data here (to avoid repeating this step)

trainData = []
trainLabel = []
testData = []
testLabel = []

k = 0

trainFile = open('train250.csv', newline='')
testFile = open('test250.csv', newline='')
trainReader = csv.reader(trainFile, delimiter=' ')
testReader = csv.reader(testFile, delimiter=' ')
    
for row in trainReader:
    if (k>0):
        data = row[0].split(',')
        image = io.imread(data[0], as_gray=True)
        trainData.append(transform.resize(image,(128,128)))
        trainLabel.append(data[1])
    else:
        k+=1

k = 0
for row in testReader:
    if (k>0):
        data = row[0].split(',')
        image = io.imread(data[0])
        testData.append(transform.resize(image,(128,128)))
        testLabel.append(data[1])
    else:
        k+=1


## 2. Apply a bank of filters and generate features for training, testing and save the dataset

In [ ]:
#. Using the overcomplete bank of filters F provided, convolve your input image with each filter in the bank
#  (make sure you check the dimensions of the returned array)
#

def getAbsFilterResponses( F, imggray ):
    
    rows, cols = imggray.shape
    num_filters = F.shape[2]
    responses = np.zeros((rows, cols, num_filters))   
    
    #. Convolve the input image with every filter in the bank of filters 
    #   to get a response array; take the absolute value of the values in the array

    for i in range(num_filters):
        filter = F[:,:,i]
        image = convolve(imggray, filter)
        responses[:,:,i] = image
    
    img_responses = np.absolute(responses)
    return img_responses


In [ ]:
def apply_filter_bank( df, label, desc ):
    print(desc)
    F = makeLMfilters() # This creates the over-complete bank of filters F
    X = np.array([])    # Initialize the numpy array to stack the responses for each image
    y = []              # The label for each image
    
    length = len(df)
    Xtemp = []

    for i in range(length):
      image = df[i]
      x = getAbsFilterResponses(F, image)
      Xtemp.append(x)
      y = [label[i] for i in range(0,length)]
    
    X = np.array(Xtemp)
    return X, y

In [ ]:
## generate training and testing features, save them in a pickle with joblib
# NOTE: The code in this cell has been written for you
# NOTE2: This process can take a VERY LONG TIME...
#

X_train, y_train = apply_filter_bank( trainData, trainLabel, 'Training -> ' )
X_test, y_test = apply_filter_bank( testData, testLabel, 'Testing -> ' )

Training -> 
Testing -> 


In [ ]:
# save the dataset, pickle using joblib
dataset = f'data.pkl'
obj = {
    'train': {
        'features': X_train,
        'label': y_train
    },
    'test': {
        'features': X_test,
        'label': y_test
    }
}
print(X_train.shape)
print(X_test.shape)
with open( dataset, 'wb' ) as f:
    joblib.dump( obj, f, compress=3 )

(250, 128, 128, 48)
(250, 128, 128, 48)


Load back the dataset (optional)
## 3. Run Kmeans with k=500 (recommended)
save the model again (optional)

In [ ]:
# load back the dataset (optional)
# NOTE: The code in this cell has been written for you
dataset = f'data.pkl'
with open( dataset, 'rb' ) as f:
    obj = joblib.load( f )
print ( 'finished loading pkl file back')
    
X_train, y_train = obj['train']['features'], obj['train']['label']
X_test, y_test = obj['test']['features'], obj['test']['label']
print( f'X_train shape: {X_train.shape}; y_train shape: {len(y_train)}; X_test shape: {X_test.shape}; y_test shape: {len(y_test)}' )

finished loading pkl file back
X_train shape: (250, 128, 128, 48); y_train shape: 250; X_test shape: (250, 128, 128, 48); y_test shape: 250


In [ ]:
# Vectorize the training set 
# NOTE: The vectorization code has been written for you below
reshape_X_train = X_train.reshape( -1, 48 )
print( f'Training dataset shape: {reshape_X_train.shape}' )

# normalize the the reshaped data (recommended to use MinMaxScaler)

scaler = MinMaxScaler()
scaler.fit(reshape_X_train)
scaled = scaler.transform(reshape_X_train)


# run kmeans (because the data is large, recommended to run minibatch kmeans)
# NOTE: This process can take a VERY LONG TIME...

kmeans = MiniBatchKMeans(n_clusters=500, batch_size=300).fit(scaled)

# save kmeans model (optional)
# NOTE: The data saving code has been written for you below
model = f'kmeans_model.pkl'
with open( model, 'wb' ) as f:
    joblib.dump( kmeans, f, compress=3 )
print( f'Kmeans model saved.' )

Training dataset shape: (4096000, 48)
Kmeans model saved.


## 4. Generate the frequency histogram (feature) for each image

In [ ]:
# load the model (optional)
model = f'kmeans_model.pkl'
with open( model, 'rb' ) as f:
    kmeans = joblib.load( f )

In [ ]:
# Given an image response (already filtered) and the kmeans model, compute its frequency histogram
# a) Vectorize the response image
# b) Max-min normalize the response
# c) Predict clusters for each 48-dimensional pixel using the k-means predict function
# d) Generate the frequency histogram by incrementing the cluster centers where each pixel belongs
# e) Normalize the resulting frequency histogram so that it sums to 1
def calc_freq( img, kmeans ):

    reshape_img = img.reshape( -1, 48 )

    scaled = scaler.transform(reshape_img)
    
    y = kmeans.predict(scaled)
    hist = np.zeros(500)

    for i in range(len(y)):
      hist[y[i]] += 1
    
    img_hist= [float(i)/sum(hist) for i in hist]
    return img_hist #this is the histogram feature representing an image

In [ ]:
# loop through the set and call calc_freq on each image in the set (training and testing)

histTrain = []
histTest = []

print("Generating histogram for train set...")
for image in X_train:
    hist = calc_freq(image, kmeans)
    histTrain.append(hist)

print("Generating histogram for test set...")
for image in X_test:
    hist = calc_freq(image, kmeans)
    histTest.append(hist)

Generating histogram for train set...
Generating histogram for test set...


## 5. Implement the "nearest neighbor classifier" by comparing each image in Testing with the reference training images to generate a label.
Compare using chi-square distance measure

In [ ]:
# Implement a function to calculate chi-square distance between 2 histograms ref and img
#
def calc_chi_sq( ref, img ):
    length = len(ref)
    dist = 0
    for i in range(length):
        numerator = (ref[i] - img[i])**2
        denominator = ref[i] + img[i]
        if denominator != 0:
            dist += (numerator/denominator)
    
    dist = 0.5*dist    
    
    return dist

In [ ]:
# Using the chi-sq distances between each img in training
# and each image in test, find the closest match for each test img in training, 
#  assign the label of the closest match to that test img
# Your output will be 'y_pred', the array of predicted test labels.

y_pred = []
for i in range(len(histTest)):
    testImage = histTest[i]
    minDist = 999999
    yPred = -1
    for j in range(len(histTrain)):
        trainImage = histTrain[j]
        dist = calc_chi_sq(testImage, trainImage)
        if dist < minDist:
            minDist = dist
            yPred = y_train[j]
    y_pred.append(yPred)

## 6. Generate a class confusion matrix and print out your accuracy specifically

In [ ]:
# Using y_test (true labels on the test data) and y_pred (the predicted labels), 
#  call the appropriate library function to display your confusion matrix, 
#  ensuring that the labels corectly match up with the folder names given

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

     01beach       0.72      0.52      0.60        25
    02forest       0.50      0.60      0.55        25
  03mountain       0.30      0.24      0.27        25
      04city       0.35      0.24      0.29        25
    05suburb       0.54      0.52      0.53        25
    06street       0.59      0.64      0.62        25
   07bedroom       0.38      0.32      0.35        25
   08kitchen       0.35      0.36      0.35        25
09livingroom       0.32      0.40      0.36        25
     10store       0.36      0.52      0.43        25

    accuracy                           0.44       250
   macro avg       0.44      0.44      0.43       250
weighted avg       0.44      0.44      0.43       250



In [ ]:
length = len(y_pred)
count = 0

for i in range(length):
  if(y_pred[i] == y_test[i]):
    count += 1

print(count/length)

0.436
